## Exploring Ebay Car Sales: Dataquest Guided Project

In [42]:
import pandas as pd
import numpy as np

autos = pd.read_csv('autos.csv', encoding = 'Latin-1')

The info and sample from the autoset reveals some areas that require cleaning.
    - all values are object or int64 type. Some categories will need to be converted from object into float types, for example the price column. 
    - There are null values in 5 columns that will need to be removed or filled. 
    - 50,000 rows are available for 20 columns in this sample which is a managable amount. 
    - Column names are camelcase instead of snakecase which needs to be aknowledged when calling and renaming columns. 

In [43]:
autos.info()
autos.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
dateCrawled            371528 non-null object
name                   371528 non-null object
seller                 371528 non-null object
offerType              371528 non-null object
price                  371528 non-null int64
abtest                 371528 non-null object
vehicleType            333659 non-null object
yearOfRegistration     371528 non-null int64
gearbox                351319 non-null object
powerPS                371528 non-null int64
model                  351044 non-null object
kilometer              371528 non-null int64
monthOfRegistration    371528 non-null int64
fuelType               338142 non-null object
brand                  371528 non-null object
notRepairedDamage      299468 non-null object
dateCreated            371528 non-null object
nrOfPictures           371528 non-null int64
postalCode             371528 non-null int64
lastSeen              

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


First the column names are replaced from camelcase to snakecase which is the standard for pandas. 

In [44]:
autos = autos.rename({'dateCrawled':'date_crawled', 'name':'name', 'seller':'seller', 
             'offerType':'offer_type', 'price':'price', 
             'abtest':'abtest', 'vehicleType':'vehicle_type', 
             'yearOfRegistration':'registration_year', 'gearbox':'gearbox', 
             'powerPS':'power_PS', 'model':'model', 'odometer':'odometer', 
             'monthOfRegistration':'registration_month', 'fuelType':'fuel_type',
             'brand':'brand', 'notRepairedDamage':'unrepaired_damage', 
             'dateCreated':'ad_created', 'nrOfPictures':'nr_of_pictures',
             'postalCode':'postal_code', 'lastSeen':'last_seen'}, axis=1)

autos['price'] = autos['price'].astype(str)
type(autos['price'])

pandas.core.series.Series

In [45]:
autos['price'] = autos['price'].str.replace('$','')
autos['price'] = autos['price'].str.replace(',','')
autos['price'] = autos['price'].astype(int)

autos['odometer'] = autos['odometer'].str.replace('km','')
autos['odometer'] = autos['odometer'].str.replace(',','')
autos['odometer'] = autos['odometer'].astype(int)
autos = autos.rename({'odometer':'odometer_km'}, axis=1)

KeyError: 'odometer'

The `describe()` function shows the following cleaning tasks that need to be carried out to make the dataset free of significant mistakes:
    - The odometer and price columns are not included in the describe function because they are strings. They must have string characters removed and be converted to intergers. 
    - price ranges are not realistic and will have outliers removed. 

In [ ]:
autos.describe()

In [ ]:
autos['price'].describe()

In [ ]:
autos_bool = autos['price'].between(500, 250000,inclusive=True)
autos = autos[autos_bool]

In [ ]:
autos['price'].describe()

Registration year has also been found to have outliers like year 1000 and 9999 so we will remove these outliers in the same way as was done for prices. The acceptable range will be in the years 1950 to 2020. 

In [ ]:
autos_bool = autos['registration_year'].between(1950, 2020,inclusive=True)
autos = autos[autos_bool]

In [ ]:
def organize_dates(date_col):
    clean_date = date_col.str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)
    return clean_date

date_crawled = organize_dates(autos['date_crawled'])
ad_created = organize_dates(autos['ad_created'])
last_seen = organize_dates(autos['last_seen'])

In [ ]:
print(date_crawled.describe())
print()
print(ad_created.describe())
print()
print(last_seen.describe())

The above values desribe basic statistics about three date columns that relate to ad posting and veiwing. There distribution is not abnormal which reflects a clean dataset but the importance of these distributions is how they reflect demand. 

The `last_seen` column showes the time when the most ads were viewed. There was an enormous spike in demand on April 4th, 5th, and  6th collectively holding almost 50% of the views in that months period. 

The vast majority of ads were created in the two weeks preceding the 3 day spike in views. Reasons for this spike in demand are unknown but it reveals an optimal time for advertising if this is a seasonal demand shift.

Different brands have different properties that can be isolated. For example, the relative price of different brands for similar cars is quite different. Next we will isolate the most popular selling brands on Ebay and then find the average price for each of those brands. Results can are in the `brands_price` dictionary

In [ ]:
top_brands = (autos['brand'].value_counts())[:15].index
top_brands

In [ ]:
brands_price = {}

for brand in top_brands:
    mean_price = autos.loc[autos['brand'] == brand, 'price'].mean()
    brands_price[brand] = mean_price

# brands_price = sorted(dict.items(brands_price), key=lambda x: -x[1])
# I took this sorting code from: https://stackoverflow.com/questions/35624064/sorting-dictionary-descending-in-python
brands_price

In [ ]:
distance = autos['odometer_km'].unique()

In [ ]:
brands_dist = {}

for brand in top_brands:
    mean_dist = autos.loc[autos['brand'] == brand, 'odometer_km'].mean()
    brands_dist[brand] = mean_dist
    
# brands_dist = sorted(dict.items(brands_dist), key=lambda x: -x[1])
print(brands_dist)

In [ ]:
price_series = pd.Series(brands_price)
dist_series = pd.Series(brands_dist)

print(price_series)
print(dist_series)

In [ ]:
price_dist = pd.DataFrame({'mean_distance':dist_series, 'mean_price':price_series})
price_dist.sort_values(by=['mean_price'], ascending=False)

In [ ]:
import matplotlib.pyplot as plt
import pylab

fig, ax = plt.subplots()
prices = price_dist['mean_price']
distances = price_dist['mean_distance']
ax.scatter(prices, distances)

ax.set_xlabel('Car Price')
ax.set_ylabel('Km Driven')
plt.show()